In [ ]:
!pip install psycopg2-binary
!pip install pandas

In [1]:
import psycopg2
import pandas as pd

In [2]:
def send_sql_query(query):
    params = {
        'database': 'focus_start',
        'host': 'localhost',
        'user': 'focus',
        'password': 'start'
    }
    conn = psycopg2.connect(**params)
    raw_data = pd.read_sql_query(query, conn)
    conn.close()
    return raw_data

# Part 1

In [3]:
send_sql_query("""
SELECT * 
FROM measurement_results 
LIMIT 10
""")

,building_id,meter,timestamp_measurement,meter_reading
0,1363,2,2016-11-25 19:00:00,156.3920
1,1364,0,2016-11-25 19:00:00,38.7500
2,1364,2,2016-11-25 19:00:00,1135.6600
3,1365,0,2016-11-25 19:00:00,238.8750
4,1366,0,2016-11-25 19:00:00,39.1000
5,1366,1,2016-11-25 19:00:00,3.2983
6,1366,2,2016-11-25 19:00:00,405.1670
7,1367,0,2016-11-25 19:00:00,115.6250
8,1367,1,2016-11-25 19:00:00,37.3632
9,1367,2,2016-11-25 19:00:00,422.2500


In [ ]:
""" ТИПОВАЯ КОНСТРУКЦИЯ

SELECT DISTINCT поле(я)_по_которому_выбираются_уникальные
FROM имя_таблицы
WHERE условие
ORDER BY поля_для_сортировки DESC ключевое_слово_для_определения_порядка_сортировки
"""

In [4]:
send_sql_query("""
SELECT DISTINCT meter
FROM measurement_results
-- WHERE meter in (0, 2)
WHERE meter BETWEEN 0 AND 2
ORDER BY meter DESC
LIMIT 10
""")

,meter
0,2
1,1
2,0


In [5]:
# _ любой символ
# %edu слова, которые заканчиваются на edu
# edu% слова, которые начинаются на edu
# %edu% любое вхождение в слово
send_sql_query("""
SELECT primary_use
FROM building_metadata
WHERE lower(primary_use) LIKE 'e_u%'
LIMIT 10
""")

,primary_use
0,Education
1,Education
2,Education
3,Education
4,Education
5,Education
6,Education
7,Education
8,Education
9,Education


In [6]:
"""ГРУППИРОВКА

SELECT 
    список_полей,
    функции_для_группировки,
FROM имя_таблицы
GROUP BY имя_полей_по_которым_производится_группировка
HAVING условие_аналог_WHERE_для_группировки
"""

'ГРУППИРОВКА\n\nSELECT \n    список_полей,\n    функции_для_группировки,\nFROM имя_таблицы\nGROUP BY имя_полей_по_которым_производится_группировка\nHAVING условие_аналог_WHERE_для_группировки\n'

In [7]:
send_sql_query("""
SELECT 
    date_part('day', timestamp_measurement), 
    COUNT(timestamp_measurement)
FROM measurement_results
GROUP BY date_part('day', timestamp_measurement)
HAVING date_part('day', timestamp_measurement) < 15
""")

,date_part,count
0,1.0,53440
1,2.0,50676
2,3.0,50662
3,4.0,50678
4,5.0,50706
5,6.0,53753
6,7.0,56704
7,8.0,56678
8,9.0,56664
9,10.0,56739


In [ ]:
"""ПОДЗАПРОС

SELECT список_полей FROM (SELECT ....) AS имя_подзапроса
"""

In [8]:
send_sql_query("""
SELECT * FROM measurement_results
WHERE meter = 0 
AND meter_reading > (
    SELECT 
        AVG(meter_reading) 
    FROM measurement_results
    WHERE meter = 0
)
""")

,building_id,meter,timestamp_measurement,meter_reading
0,1365,0,2016-11-25 19:00:00,238.875
1,1368,0,2016-11-25 19:00:00,185.800
2,1378,0,2016-11-25 19:00:00,762.300
3,1380,0,2016-11-25 19:00:00,395.525
4,1382,0,2016-11-25 19:00:00,411.850
...,...,...,...,...
295143,1382,0,2016-12-31 23:00:00,467.000
295144,1391,0,2016-12-31 23:00:00,188.825
295145,1432,0,2016-12-31 23:00:00,403.450
295146,1437,0,2016-12-31 23:00:00,195.925


# Part 2

In [ ]:
"""
measurement_results.csv

building_id - внешний ключ для метаданных здания.
meter - Идентификационный код счетчика. Читается как {
        0: электричество, 
        1: охлажденная вода, 
        2: пар, 
        3: горячая вода}. 
Не в каждом здании есть все типы счетчиков.
timestamp_measurement - когда было проведено измерение
meter_reading - целевая переменная. Потребление энергии в кВтч (или эквивалент). 

building_metadata.csv

site_id - Внешний ключ для файлов погоды.
building_id - Внешний ключ для training.csv
primary_use - индикатор основной категории деятельности для здания на основе определений типов свойств EnergyStar
square_feet - Общая площадь здания
year_built - Год открытия здания
floor_count - Этажность здания

weather_train.csv
Данные о погоде с метеостанции как можно ближе к объекту.

site_id
air_temperature - Градусы Цельсия
cloud_coverage - Часть неба, покрытая облаками, в октах
dew_temperature - Градусы Цельсия
pred_depth_1_hr - Миллиметры
sea_level_pressure - Миллибар / гектопаскалях
wind_direction - направление по компасу (0-360)
wind_speed - Метров в секунду
timestamp_measurement
"""

In [ ]:
"""КОНСТРУКЦИЯ UNION

SELECT ....
UNION
SELECT ....
"""

In [9]:
send_sql_query("""
SELECT DISTINCT(meter) 
FROM (
    SELECT * FROM measurement_results WHERE meter = 0
    UNION
    SELECT * FROM measurement_results WHERE meter = 1) AS filters
""")

,meter
0,0
1,1


In [ ]:
"""КОНСТРУКЦИЯ WITH

WITH
имя_временной_таблицы AS (SELECT ....),
имя_временной_таблицы2 AS (SELECT ....)
SELECT .... c имя_временной_таблицы и имя_временной_таблицы2
"""

In [ ]:
send_sql_query("""
WITH
avg_table as (
    SELECT
        meter,
        building_id,
        AVG(meter_reading) 
    FROM measurement_results
    GROUP BY building_id, meter
),
buildings as (
    SELECT * 
    FROM measurement_results as mr
    LEFT JOIN (
            SELECT 
                building_id as building_id_meta,
                year_built
            FROM building_metadata
            ) as bm 
    ON mr.building_id = bm.building_id_meta 
)
SELECT * 
FROM buildings as bld
LEFT JOIN avg_table as at ON bld.building_id = at.building_id AND bld.meter = at.meter
WHERE bld.year_built < 1940
""")

In [ ]:
"""
WITH
основная_таблица_с_таргетом_и_идентификаторами as (SELECT 
                                                        идентификатор_здания, 
                                                        тип_счетчика,
                                                        целевая_переменная
                                                    FROM ....),
таблица_где_насчитываем_признак1 as (SELECT ....),
таблица_где_насчитываем_признак2 as (SELECT ....)
SELECT
    идентификатор_здания, 
    тип_счетчика,
    целевая_переменная,
    признак1,
    признак2
FROM основная_таблица_с_таргетом_и_идентификаторами
LEFT JOIN таблица_где_насчитываем_признак1 ON ....
LEFT JOIN таблица_где_насчитываем_признак2 ON ....
"""

# Part 3

In [10]:
""" ОБЩИЙ СИНТАКСИС ОКОННОЙ ФУНКЦИИ

SELECT
Название функции (столбец для вычислений) 
OVER (
      PARTITION BY столбец для группировки
      ORDER BY столбец для сортировки
      ROWS или RANGE выражение для ограничения строк в пределах группы
      )
"""

' ОБЩИЙ СИНТАКСИС ОКОННОЙ ФУНКЦИИ\n\nSELECT\nНазвание функции (столбец для вычислений) \nOVER (\n      PARTITION BY столбец для группировки\n      ORDER BY столбец для сортировки\n      ROWS или RANGE выражение для ограничения строк в пределах группы\n      )\n'

In [13]:
# Агрегатные функции

send_sql_query("""
SELECT 
    meter,
    timestamp_measurement,
    meter_reading,
    AVG(meter_reading) 
        OVER(
            PARTITION BY meter
            ORDER BY timestamp_measurement
        ) as average_meter_reading
FROM measurement_results
ORDER BY (building_id, timestamp_measurement)
LIMIT 20
""")

,meter,timestamp_measurement,meter_reading,average_meter_reading
0,0,2016-11-25 20:00:00,209.546,158.446295
1,0,2016-11-25 21:00:00,220.467,158.162729
2,0,2016-11-25 22:00:00,234.801,156.788665
3,0,2016-11-25 23:00:00,228.658,154.783503
4,0,2016-11-26 00:00:00,233.436,153.171200
5,0,2016-11-26 01:00:00,230.705,152.011735
6,0,2016-11-26 02:00:00,231.388,151.131054
7,0,2016-11-26 03:00:00,236.848,150.428155
8,0,2016-11-26 04:00:00,234.801,149.798513
9,0,2016-11-26 05:00:00,236.166,149.479989


In [ ]:
# Подсказка!

send_sql_query("""
SELECT building_id, meter, meter_reading, timestamp_measurement
    FROM measurement_results
    ORDER BY (building_id, timestamp_measurement)
    LIMIT 20
""")

In [14]:
# Ранжирующие функции

send_sql_query("""
SELECT
    timestamp_measurement, building_id, 
    ROW_NUMBER() OVER(PARTITION BY timestamp_measurement ORDER BY building_id) AS row_number, 
    RANK() OVER(PARTITION BY timestamp_measurement ORDER BY building_id) AS rank,
    DENSE_RANK() OVER(PARTITION BY timestamp_measurement ORDER BY building_id) AS dense_rank,
    NTILE(2) OVER(PARTITION BY timestamp_measurement ORDER BY building_id) AS ntile
FROM (SELECT building_id, meter, meter_reading, timestamp_measurement
FROM measurement_results
LIMIT 20) AS test
""")

,timestamp_measurement,building_id,row_number,rank,dense_rank,ntile
0,2016-11-25 19:00:00,1363,1,1,1,1
1,2016-11-25 19:00:00,1364,2,2,2,1
2,2016-11-25 19:00:00,1364,3,2,2,1
3,2016-11-25 19:00:00,1365,4,4,3,1
4,2016-11-25 19:00:00,1366,5,5,4,1
5,2016-11-25 19:00:00,1366,6,5,4,1
6,2016-11-25 19:00:00,1366,7,5,4,1
7,2016-11-25 19:00:00,1367,8,8,5,1
8,2016-11-25 19:00:00,1367,9,8,5,1
9,2016-11-25 19:00:00,1367,10,8,5,1


In [15]:
# Функции смещения

send_sql_query("""
SELECT
    building_id, meter,
    LAG(building_id) OVER(PARTITION BY meter ORDER BY meter) AS lag,
    LEAD(building_id) OVER(PARTITION BY meter ORDER BY meter) AS lead, 
    FIRST_VALUE(building_id) OVER(PARTITION BY meter ORDER BY meter) AS first,
    LAST_VALUE(building_id) OVER(PARTITION BY meter ORDER BY meter) AS last
FROM (SELECT building_id, meter
FROM measurement_results
LIMIT 20) AS test
""")

,building_id,meter,lag,lead,first,last
0,1375,0,NaN,1364.0,1375,1371
1,1364,0,1375.0,1365.0,1375,1371
2,1365,0,1364.0,1366.0,1375,1371
3,1366,0,1365.0,1367.0,1375,1371
4,1367,0,1366.0,1368.0,1375,1371
5,1368,0,1367.0,1369.0,1375,1371
6,1369,0,1368.0,1370.0,1375,1371
7,1370,0,1369.0,1371.0,1375,1371
8,1371,0,1370.0,NaN,1375,1371
9,1374,1,NaN,1366.0,1374,1367


In [16]:
# Аналитические функции

send_sql_query("""
SELECT
    building_id, meter,
    CUME_DIST() OVER(PARTITION BY meter ORDER BY building_id) AS cume_dist,
    PERCENT_RANK() OVER(PARTITION BY meter ORDER BY building_id) AS percent_rank
FROM (SELECT building_id, meter
FROM measurement_results
LIMIT 20) AS test
""")

,building_id,meter,cume_dist,percent_rank
0,1364,0,0.111111,0.000
1,1365,0,0.222222,0.125
2,1366,0,0.333333,0.250
3,1367,0,0.444444,0.375
4,1368,0,0.555556,0.500
5,1369,0,0.666667,0.625
6,1370,0,0.777778,0.750
7,1371,0,0.888889,0.875
8,1375,0,1.000000,1.000
9,1366,1,0.200000,0.000


In [17]:
# Кастомные функции

send_sql_query("""
SELECT * FROM is_more_average(0, 0, 15);
""")

,is_more_average
0,False


In [18]:
send_sql_query("""
WITH
test as (SELECT building_id, meter, meter_reading, timestamp_measurement
    FROM measurement_results
    ORDER BY (building_id, timestamp_measurement)
    LIMIT 20)
SELECT is_more_average(test.building_id, test.meter, test.meter_reading) FROM test;
""")

,is_more_average
0,True
1,True
2,True
3,True
4,True
5,True
6,True
7,True
8,True
9,True
